# Ofertas Promocionais de Companhias Aéreas Brasileiras

Este notebook faz web scraping nos sites das principais companhias aéreas brasileiras (Azul, Gol e LATAM),
extrai ofertas promocionais usando a API da OpenAI e gera um relatório formatado em Markdown.

**Pré-requisitos:**
- Chave `OPENAI_API_KEY` configurada no arquivo `.env` na raiz do projeto
- Playwright instalado (veja a próxima célula)

In [ ]:
# Descomente e execute na primeira vez para instalar o Playwright:
# !uv add playwright
# !playwright install chromium

In [1]:
import os
import asyncio
import nest_asyncio
from datetime import datetime
from dotenv import load_dotenv
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display

nest_asyncio.apply()

In [ ]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("Chave de API não encontrada! Verifique o arquivo .env na raiz do projeto.")
elif api_key.strip() != api_key:
    print("A chave de API contém espaços extras no início ou fim — remova-os do .env.")
else:
    print("Chave de API encontrada com sucesso!")

client = OpenAI()

Chave de API encontrada com sucesso!


In [13]:
COMPANHIAS = {
    "Gol": "https://www.voegol.com.br/nh/",
    "LATAM": "https://www.latamairlines.com/br/pt",
}

## Estratégia de Scraping

Os sites das companhias aéreas brasileiras são construídos com JavaScript pesado (React, Angular, etc.)
e retornam erro 403 quando acessados com `requests` simples. Por isso usamos o **Playwright**,
que executa um navegador headless real (Chromium), renderiza o JavaScript e nos dá o HTML final.

Usamos a API **async** do Playwright (com `nest_asyncio` para compatibilidade com Jupyter) porque
o Jupyter já roda dentro de um event loop asyncio, o que impede o uso da API sync.

A estratégia:
- Aguardar `networkidle` (nenhuma requisição de rede por 500ms)
- Esperar 3 segundos extras para banners e carrosséis de promoções carregarem
- Limpar o HTML removendo tags desnecessárias (scripts, estilos, navegação)

In [14]:
MAX_CHARS = 8000

USER_AGENT = (
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/131.0.0.0 Safari/537.36"
)

TAGS_TO_REMOVE = [
    "script", "style", "nav", "footer", "header",
    "img", "svg", "form", "button", "noscript",
]


async def scrape_airline(url: str, airline_name: str) -> str | None:
    """Acessa o site da companhia aérea com Playwright e retorna o texto limpo."""
    print(f"  Acessando {airline_name} ({url})...")
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(
                headless=True,
                channel="chrome",
                args=["--disable-blink-features=AutomationControlled"],
            )
            context = await browser.new_context(
                user_agent=USER_AGENT,
                viewport={"width": 1920, "height": 1080},
                locale="pt-BR",
                timezone_id="America/Sao_Paulo",
            )
            page = await context.new_page()

            # Remover flag que indica automação
            await page.add_init_script(
                "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
            )

            await page.goto(url, wait_until="domcontentloaded", timeout=30000)
            # Esperar o body e dar tempo para JS renderizar conteúdo dinâmico
            await page.wait_for_selector("body")
            await page.wait_for_timeout(5000)
            html = await page.content()
            await browser.close()

        soup = BeautifulSoup(html, "html.parser")
        for tag in TAGS_TO_REMOVE:
            for element in soup.find_all(tag):
                element.decompose()

        text = soup.get_text(separator="\n", strip=True)
        print(f"  {airline_name}: {len(text)} caracteres extraídos")
        return text[:MAX_CHARS]

    except Exception as e:
        print(f"  Erro ao acessar {airline_name}: {e}")
        return None

In [15]:
print("Iniciando scraping dos sites...\n")

scraped_data = {}
extraction_time = datetime.now()

for name, url in COMPANHIAS.items():
    scraped_data[name] = await scrape_airline(url, name)

print("\n--- Resumo ---")
for name, content in scraped_data.items():
    status = f"{len(content)} chars" if content else "FALHOU"
    print(f"  {name}: {status}")

Iniciando scraping dos sites...

  Acessando Gol (https://www.voegol.com.br/nh/)...
  Gol: 10865 caracteres extraídos
  Acessando LATAM (https://www.latamairlines.com/br/pt)...
  LATAM: 4397 caracteres extraídos

--- Resumo ---
  Gol: 8000 chars
  LATAM: 4397 chars


## Análise com LLM

Agora usamos a API da OpenAI para analisar o conteúdo extraído de cada site.
O modelo recebe um **system prompt** com instruções específicas para extrair ofertas
em formato estruturado, e um **user prompt** com o conteúdo do site.

In [16]:
system_prompt = """Você é um analista especializado em ofertas de passagens aéreas brasileiras.
Sua tarefa é extrair e organizar as ofertas promocionais encontradas no conteúdo de sites
de companhias aéreas. Responda sempre em português brasileiro.

Regras:
- Foque apenas em ofertas, promoções e preços de passagens
- Ignore textos de navegação, menus, rodapés e conteúdo institucional
- Se não encontrar ofertas claras, indique o que foi possível identificar sobre promoções
- Use formato Markdown na resposta
- Não invente dados — se uma informação não estiver disponível, escreva "não informado"
"""


def build_user_prompt(airline_name: str, content: str) -> str:
    return f"""Analise o conteúdo abaixo, extraído do site da {airline_name}, e liste as ofertas
promocionais encontradas. Para cada oferta, inclua (quando disponível):

- **Destino**: cidade ou rota
- **Preço**: valor em reais
- **Período**: datas da promoção ou da viagem
- **Condições**: restrições, milhas, parcelamento, etc.

Se alguma informação não estiver disponível, use "não informado".

Conteúdo do site:

{content}
"""

In [17]:
def analyze_airline_offers(airline_name: str, content: str | None) -> str:
    """Envia o conteúdo extraído para o LLM e retorna a análise de ofertas."""
    if content is None:
        return (
            f"Não foi possível acessar o site da {airline_name}. "
            "O site pode estar bloqueando acesso automatizado ou fora do ar."
        )

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            temperature=0.2,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": build_user_prompt(airline_name, content)},
            ],
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Erro ao analisar ofertas da {airline_name}: {e}"

In [18]:
print("Analisando ofertas com LLM...\n")

analyses = {}
for name, content in scraped_data.items():
    print(f"  Analisando {name}...")
    analyses[name] = analyze_airline_offers(name, content)
    print(f"  {name}: concluído")

print("\nAnálise finalizada!")

Analisando ofertas com LLM...

  Analisando Gol...
  Gol: concluído
  Analisando LATAM...
  LATAM: concluído

Análise finalizada!


In [19]:
date_str = extraction_time.strftime("%d/%m/%Y às %H:%M")
date_file = extraction_time.strftime("%Y-%m-%d")

sections = []
for name in COMPANHIAS:
    sections.append(f"## {name}\n\n{analyses[name]}")

full_report = f"""# Ofertas de Passagens Aéreas — Companhias Brasileiras

**Data da consulta:** {date_str}

{chr(10).join(sections)}

---

*Relatório gerado automaticamente. Preços e disponibilidade sujeitos a alteração.
Consulte os sites oficiais para informações atualizadas.*
"""

display(Markdown(full_report))

# Salvar como arquivo .md
output_dir = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(output_dir, f"ofertas_aereas_{date_file}.md")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_report)

print(f"\nRelatório salvo em: {output_path}")

# Ofertas de Passagens Aéreas — Companhias Brasileiras

**Data da consulta:** 22/02/2026 às 13:06

## Gol

```markdown
# Ofertas Promocionais Encontradas no Site da Gol

1. **Destino:** Natal - Montevidéu  
   **Preço:** não informado  
   **Período:** a partir de 21/03 (ano não informado)  
   **Condições:** Consulte condições; voo sem escalas; pontualidade e conforto da GOL.

2. **Destino:** Bariloche  
   **Preço:** não informado  
   **Período:** a partir de junho/2026  
   **Condições:** Voo sem escala com a GOL.

3. **Destino:** Rotas nacionais e internacionais diversas (Argentina destacada)  
   **Preço:** não informado  
   **Período:** a partir de julho/2026  
   **Condições:** Voo sem escalas; pontualidade e conforto da GOL.

4. **Condições gerais de pagamento:**  
   - Parcelamento em até 10 vezes no boleto parcelado PaGOL.  
   - Programa de fidelidade Smiles para acumular milhas.  
   - Cartão de crédito GOL Smiles com benefícios exclusivos.

---

# Observações

- Não foram encontrados preços específicos para as passagens.  
- As promoções destacam novas rotas e facilidades de pagamento, mas sem valores promocionais explícitos.  
- Recomenda-se consultar diretamente as condições específicas no site da Gol para detalhes completos.
```
## LATAM

# Ofertas promocionais encontradas no site da LATAM Brasil

---

### Oferta 1
- **Destino:** Florianópolis (voo direto saindo de Porto Alegre)
- **Preço:** R$ 153,36 (somente ida)
- **Período:** 24/03/2026
- **Condições:** Classe Economy, acumula milhas, taxas incluídas

---

### Oferta 2
- **Destino:** São Paulo (voo direto saindo de Porto Alegre)
- **Preço:** R$ 216,36 (somente ida)
- **Período:** 17/03/2026
- **Condições:** Classe Economy, acumula milhas, taxas incluídas

---

### Oferta 3
- **Destino:** São Paulo (voo direto saindo de Porto Alegre)
- **Preço:** R$ 281,36 (somente ida)
- **Período:** 22/03/2026
- **Condições:** Classe Economy, acumula milhas, taxas incluídas

---

### Oferta 4
- **Destino:** Belo Horizonte (voo direto saindo de Porto Alegre)
- **Preço:** R$ 292,36 (somente ida)
- **Período:** 17/03/2026
- **Condições:** Classe Economy, acumula milhas, taxas incluídas

---

### Oferta 5
- **Destino:** Rio de Janeiro (voo direto saindo de Porto Alegre)
- **Preço:** R$ 342,36 (somente ida)
- **Período:** 22/03/2026
- **Condições:** Classe Economy, acumula milhas, taxas incluídas

---

### Oferta 6 - Pacote Rio de Janeiro
- **Destino:** Rio de Janeiro (voo direto + hotel)
- **Preço:** R$ 1.216,00 por pessoa
- **Período:** 27/03/2026 a 02/04/2026 (6 noites)
- **Condições:** Voo direto saindo de Porto Alegre, hotel Atlântico Business Centro, acumula milhas, inclui impostos e taxas

---

### Oferta 7 - Pacote Santiago do Chile
- **Destino:** Santiago do Chile (voo com conexão + hotel)
- **Preço:** R$ 2.713,00 por pessoa
- **Período:** 06/06/2026 a 12/06/2026 (6 noites)
- **Condições:** Voo com conexão saindo de Porto Alegre, hotel Terrado Lyon, acumula milhas, inclui impostos e taxas

---

### Outras promoções gerais
- Descontos de até 30% em pacotes
- Acúmulo de 1,5 Milha LATAM Pass + 1 ponto qualificável a cada real gasto em pacotes
- Descontos a partir de 15% em hotéis, com acúmulo de milhas e pontos qualificáveis
- Desconto de até 10% em aluguel de carros, com acúmulo de milhas e pontos qualificáveis
- Até 30% OFF em seguros de viagem, com acúmulo de milhas e pontos qualificáveis

---

Se desejar mais detalhes ou outras ofertas, é indicado consultar diretamente o site da LATAM.

---

*Relatório gerado automaticamente. Preços e disponibilidade sujeitos a alteração.
Consulte os sites oficiais para informações atualizadas.*



Relatório salvo em: /Users/gersonazevedo/Projects/llm_engineering/week1/community-contributions/ofertas_aereas_2026-02-22.md


## Notas e Limitações

- **Preços mudam constantemente** — os valores extraídos refletem o momento da consulta
- **Proteção anti-bot** — os sites podem bloquear acessos automatizados a qualquer momento
- **Conteúdo dinâmico** — nem todas as promoções são visíveis no HTML inicial (algumas exigem interação)

### Sugestões de extensão

- Adicionar mais companhias (ex: MAP, Voepass)
- Agendar execução periódica para monitorar variações de preço
- Capturar screenshots das páginas para referência visual
- Comparar preços entre companhias para a mesma rota